### Compute viscosity from json file

This cookbook is a calculator of rheology

First, we load a rheology from file or a dict

Then we pass T, P, strain rate condition and compute the viscosities.

In [ ]:
import os
import json
import os
import sys
import numpy as np
from matplotlib import pyplot as plt

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']

sys.path.append(os.path.join(ASPECT_LAB_DIR))

import shilofue.Rheology as Rheology

#### Step 1: load the json file

Option 1: load the json file

In [ ]:
json_path = "/home/lochy/ASPECT_PROJECT/TwoDSubduction_DIR/EBA_CDPT_3dconsistent_3/eba_cdpt_SA80.0_OA40.0_width61_sc21/configurations/mantle_profile_aspect_v1_HK03_wet_mod_dEdiff-4.0000e+04_dEdisl3.0000e+04_dVdiff-5.500000e-06_dVdisl2.1200e-06_dAdiff3.3333e-01_dAdisl1.0403e+00.json"

assert(os.path.isabs(json_path))

with open(json_path, 'r') as fin:
    _dict = json.load(fin)

Option 2: Load a dictionary directly.

This example is the 300 ppm rheology I used before

In [ ]:
# import the _dict directly

_dict = {"diffusion_creep": {"A": 2.4536131281805923e-17, "d": 0.01, "n": 1.0, "m": 3.0, "E": 285000.0, "V": 6.9e-06},
"dislocation_creep": {"A": 1.3930166603371937e-16, "d": 0.01, "n": 3.5, "m": 0.0, "E": 490000.0, "V": 1.34e-05},
"diffusion_lm": {"A": 1.5663703327846732e-19, "d": 0.01, "n": 1.0, "m": 3.0, "E": 285000.0, "V": 3e-06}}

In [ ]:
diffusion_creep = _dict["diffusion_creep"]
dislocation_creep = _dict["dislocation_creep"]
peierls_creep = Rheology.GetPeierlsRheology("MK10")

Step 2: compute viscosity

In [ ]:
include_peierls = False

strain_rate = 2.44867e-15
P = 7.27096e+09
T = 1709.19


diffusion_creep_eta = Rheology.CreepRheologyInAspectViscoPlastic(diffusion_creep, strain_rate, P, T)
dislocation_creep_eta = Rheology.CreepRheologyInAspectViscoPlastic(dislocation_creep, strain_rate, P, T)
peierls_creep_eta = Rheology.PeierlsCreepRheology(peierls_creep, strain_rate, P, T)

temp = (1.0/diffusion_creep_eta + 1.0/dislocation_creep_eta)
if include_peierls:
    temp += 1.0 / peierls_creep_eta
composite_eta = 1.0 / temp

print("diffusion creep viscosity: ", diffusion_creep_eta)
print("dislocation creep viscosity: ", dislocation_creep_eta)
print("peierls creep viscosity: ", peierls_creep_eta)
print("composite viscosity (include Peierls %s): " % str(include_peierls), composite_eta)